In [ ]:
#| default_exp classes.DomoGrant

In [ ]:
#| export
from dataclasses import dataclass, field
from nbdev.showdoc import patch_to
import httpx

import domolibrary.client.DomoAuth as dmda
import domolibrary.utils.DictDot as util_dd
import domolibrary.routes.grant as grant_routes


In [ ]:
# | export
@dataclass
class DomoGrant:
    id: str
    display_group: str = None
    title: str = None
    depends_on_ls: list[str] = None
    description: str = None
    role_membership_ls: list[str] = field(default=None)

    def __post_init__(self):
        self.id = str(self.id)

    def __eq__(self, other):
        if not isinstance(other, DomoGrant):
            return False

        return self.id == other.id


    @classmethod
    def _from_json(cls, obj):

        dd = obj
        if not isinstance(dd, util_dd.DictDot):
            dd = util_dd.DictDot(obj)

        return cls(id=dd.authority,
                   display_group=dd.authorityUIGroup,
                   depends_on_ls=dd.dependsOnAuthorities,
                   title=dd.title,
                   description=dd.description,
                   role_membership_ls=[str(role) for role in dd.roleIds])


In [ ]:
#| export
@dataclass
class DomoGrants:
    pass



In [ ]:
# | exporti
@patch_to(DomoGrants, cls_method=True)
async def get_grants(cls: DomoGrants,
                     auth: dmda.DomoAuth,
                     session: httpx.AsyncClient = None, debug_api: bool = False, return_raw: bool = False,
                     ):
    res = await grant_routes.get_grants(auth=auth, debug_api = debug_api, session = session)

    if return_raw or not res.is_success: 
        return res
    
    return [DomoGrant._from_json(row) for index, row in enumerate(res.response)]


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

res =await DomoGrants.get_grants(auth = token_auth)
res[0:5]

[DomoGrant(id='authorization.roles', display_group='COMPANY', title='Manage all roles', depends_on_ls=['ui.admin'], description='Create, edit and delete custom roles. Assign anyone to any role in this instance.', role_membership_ls=['1', '275763436', '810756122']),
 DomoGrant(id='audit', display_group='COMPANY', title='View activity logs', depends_on_ls=['ui.admin'], description='View and export audit logs.', role_membership_ls=['1', '275763436', '810756122', '1927158482', '1563101750', '2097317660']),
 DomoGrant(id='customer.edit', display_group='COMPANY', title='Manage all company settings', depends_on_ls=['ui.admin'], description='Manage company metadata, authentication rules, authorized domain rules, export security, mobile home screen layouts, publishing, SSO Configuration, whitelist rules, report scheduler and licenses.', role_membership_ls=['1', '275763436', '810756122']),
 DomoGrant(id='developer.token.manage', display_group='COMPANY', title='Manage all access tokens and API cl

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
